In [7]:
import os
import pandas as pd
import shutil
from tqdm import tqdm

# CSV 파일 경로 및 읽어올 컬럼 이름 지정
csv_file_path = '/media/innopam/GeoAI/Merge_AE/grid/mapid_no.csv'
column_name = 'MAPIDCD_NO'  # 도엽번호 열 이름

# TIF 파일이 있는 디렉토리 경로 지정
tif_directory_path = '/media/innopam/GeoAI/04.Grid_Image/AE_2020_12/AE_2020_12_tif'

# 결과를 저장할 디렉토리 경로 지정
output_directory_path = '/media/innopam/GeoAI/Merge_AE/grid2020'

# CSV 파일 읽기
df = pd.read_csv(csv_file_path)

# tqdm을 사용하여 진행 상황을 시각화
for index, row in tqdm(df.iterrows(), total=len(df), desc="Copying TIF files"):
    doepno = str(row[column_name])[:9]  # 도엽번호 앞 9글자 추출
    tif_file_name = f'{doepno}_AE_2020_12.tif'
    tif_file_path = os.path.join(tif_directory_path, tif_file_name)
    
    # 해당하는 TIF 파일이 있는지 확인하고 있으면 결과 디렉토리로 이동
    if os.path.exists(tif_file_path):
        output_file_path = os.path.join(output_directory_path, tif_file_name)
        shutil.copy(tif_file_path, output_file_path)
    else:
        print(f'{tif_file_name} 파일을 찾을 수 없습니다.')

Copying TIF files: 100%|██████████| 99/99 [06:31<00:00,  3.96s/it]


In [13]:
#merge code
import os
import glob
from rasterio.merge import merge
from rasterio import open as rio_open

def merge_all_images(input_folder, output_path):
    # 폴더 내의 모든 이미지 파일 경로 가져오기
    image_paths = glob.glob(os.path.join(input_folder, '*.tif'))

    # 이미지 파일이 없으면 에러 메시지 출력
    if not image_paths:
        print("Error: No TIFF files found in the specified folder.")
        return

    # 이미지 열기
    src_files_to_mosaic = [rio_open(image_path) for image_path in image_paths]

    # 이미지 병합
    mosaic, out_trans = merge(src_files_to_mosaic)

    # 병합된 이미지의 메타데이터 설정
    out_meta = src_files_to_mosaic[0].meta.copy()
    out_meta.update({"driver": "GTiff",
                     "height": mosaic.shape[1],
                     "width": mosaic.shape[2],
                     "transform": out_trans})

    # 병합된 이미지 저장
    with rio_open(output_path, "w", **out_meta) as dest:
        dest.write(mosaic)

# 예제 사용법
input_folder = '/media/innopam/GeoAI/Merge_AE/grid2020'  # 이미지가 있는 폴더 경로
output_path = '/media/innopam/GeoAI/Merge_AE/grid2020/merged_image_2020.tif'  # 병합된 이미지 저장 경로

merge_all_images(input_folder, output_path)


In [1]:
import os
import glob
from rasterio.merge import merge
from rasterio import open as rio_open
from tqdm import tqdm  # Import tqdm for the progress bar

def merge_all_images(input_folder, output_path):
    # 폴더 내의 모든 이미지 파일 경로 가져오기
    image_paths = glob.glob(os.path.join(input_folder, '*.tif'))

    # 이미지 파일이 없으면 에러 메시지 출력
    if not image_paths:
        print("Error: No TIFF files found in the specified folder.")
        return

    # 이미지 열기
    src_files_to_mosaic = [rio_open(image_path) for image_path in image_paths]

    # 이미지 병합
    mosaic, out_trans = merge(src_files_to_mosaic)

    # 병합된 이미지의 메타데이터 설정
    out_meta = src_files_to_mosaic[0].meta.copy()
    out_meta.update({"driver": "GTiff",
                     "height": mosaic.shape[1],
                     "width": mosaic.shape[2],
                     "transform": out_trans})

    # 병합된 이미지 저장
    with rio_open(output_path, "w", **out_meta) as dest:
        # Use tqdm for progress bar
        for i in tqdm(range(mosaic.shape[0]), desc='Writing Merged Image'):
            dest.write(mosaic[i], indexes=i+1)

# 예제 사용법
input_folder = '/media/innopam/GeoAI/Merge_AE/grid2020'  # 이미지가 있는 폴더 경로
output_path = '/media/innopam/GeoAI/Merge_AE/grid2020/merged_image_2020.tif'  # 병합된 이미지 저장 경로

merge_all_images(input_folder, output_path)


Writing Merged Image: 100%|██████████| 3/3 [00:39<00:00, 13.21s/it]


In [1]:
import os
import glob
from rasterio.merge import merge
from rasterio import open as rio_open
from rasterio.crs import CRS
from tqdm import tqdm

def merge_all_images(input_folder, output_path):
    # 폴더 내의 모든 이미지 파일 경로 가져오기
    image_paths = glob.glob(os.path.join(input_folder, '*.tif'))

    # 이미지 파일이 없으면 에러 메시지 출력
    if not image_paths:
        print("Error: No TIFF files found in the specified folder.")
        return

    # 이미지 열기
    src_files_to_mosaic = [rio_open(image_path) for image_path in image_paths]

    # 이미지 병합
    mosaic, out_trans = merge(src_files_to_mosaic)

    # 병합된 이미지의 메타데이터 설정
    out_meta = src_files_to_mosaic[0].meta.copy()
    out_meta.update({"driver": "GTiff",
                     "height": mosaic.shape[1],
                     "width": mosaic.shape[2],
                     "transform": out_trans,
                     "crs": CRS.from_epsg(5179)})  # Set CRS to EPSG 5179

    # 병합된 이미지 저장
    with rio_open(output_path, "w", **out_meta) as dest:
        # Use tqdm for progress bar
        for i in tqdm(range(mosaic.shape[0]), desc='Writing Merged Image'):
            dest.write(mosaic[i], indexes=i+1)

# 예제 사용법
input_folder = '/media/innopam/GeoAI/Merge_AE/grid2020'  # 이미지가 있는 폴더 경로
output_path = '/media/innopam/GeoAI/Merge_AE/grid2020/merged_image_2020.tif'  # 병합된 이미지 저장 경로

merge_all_images(input_folder, output_path)


ERROR 1: PROJ: internal_proj_create_from_database: /home/innopam/anaconda3/envs/geospatial/share/proj/proj.db lacks DATABASE.LAYOUT.VERSION.MAJOR / DATABASE.LAYOUT.VERSION.MINOR metadata. It comes from another PROJ installation.


CRSError: The EPSG code is unknown. PROJ: internal_proj_create_from_database: /home/innopam/anaconda3/envs/geospatial/share/proj/proj.db lacks DATABASE.LAYOUT.VERSION.MAJOR / DATABASE.LAYOUT.VERSION.MINOR metadata. It comes from another PROJ installation.